In [1]:
import pandas as pd
from thefuzz import process
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import numpy as np

# Load the first CSV file into a DataFrame
fifa_players = pd.read_csv('male_players.csv')

combined_player_stats = pd.read_csv("filtered_combined_player_stats.csv")

/tmp/ipykernel_2871809/4223812721.py:12: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_players = pd.read_csv('male_players.csv')


In [2]:
#league_id 
#LaLiga - 53
#Prem - 13
#Bundesliga - 19
#Serie A - 31
#Ligue 1 - 16

filtered_fifa_players = fifa_players[fifa_players['league_id'].isin([53, 13, 19, 31, 16])]

fifa23_players = filtered_fifa_players[filtered_fifa_players['fifa_version'].isin([23])]
fifa23_players.head()
print(fifa23_players.shape[0])

2880


In [3]:
known_names = fifa23_players['long_name']

# Add a new column called 'rating' to the player_stats DataFrame
combined_player_stats['rating'] = None

# Loop through each corrupted name in the player_stats DataFrame
for idx, name in enumerate(combined_player_stats['Player']):
    # Find the best match from the list of known names
    best_match = process.extractOne(name, known_names)
    if best_match is not None:
        matched_name = best_match[0]
        rating = fifa23_players.loc[fifa23_players['long_name'] == matched_name, 'overall'].iloc[0]
        combined_player_stats.at[idx, 'rating'] = rating
        # Additional checks
        squad = combined_player_stats.at[idx, 'Squad']
        born_year = combined_player_stats.at[idx, 'Born']
        # Find the matched player's info
        matched_player = fifa23_players[fifa23_players['long_name'] == matched_name]
        if not matched_player.empty:
            matched_squad = matched_player['club_name'].iloc[0]
            matched_born_year = int(matched_player['dob'].iloc[0].split('-')[0])
            if born_year == matched_born_year:
                combined_player_stats.at[idx, 'rating'] = rating/99
                #print(best_match)
                #print(f"Matched name: {name}, Best match: {matched_name}, Rating: {rating}")
                #print(f"Squad and birth year matched for {name}")
            else:
                combined_player_stats.drop(idx, inplace=True)

In [4]:
# Replace NaN values with 0
combined_player_stats.fillna(0, inplace=True)
#shuffle the players
combined_player_stats = combined_player_stats.sample(frac=1).reset_index(drop=True)

# Split positions into a list
combined_player_stats['Positions_list'] = combined_player_stats['Pos'].str.split(',')

# Get unique positions
unique_positions = set(position for sublist in combined_player_stats['Positions_list'] for position in sublist)

# Create binary indicator variables for each position
for position in unique_positions:
    combined_player_stats[position] = combined_player_stats['Positions_list'].apply(lambda x: 1 if position in x else 0)

# Drop unnecessary columns
combined_player_stats.drop(['Positions_list'], axis=1, inplace=True)

combined_player_stats.head()

# Reorder the columns to make 'rating' the last column
columns_ordered = [col for col in combined_player_stats.columns if col != 'rating'] + ['rating']

# Reassign the DataFrame with the reordered columns
combined_player_stats = combined_player_stats[columns_ordered]

# Display the updated DataFrame
#combined_player_stats.head()

In [5]:
from sklearn.ensemble import RandomForestRegressor

#random forest without categorical

X = combined_player_stats.iloc[1:, 5:-1].values
Y = combined_player_stats.iloc[1:, -1].values  # Only the last column, excluding first row

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the model
model_rf = RandomForestRegressor(random_state=42)

# Fit the model
model_rf.fit(X_train, Y_train)

# Predictions
Y_pred_rf = model_rf.predict(X_test)

# Compute evaluation metrics
mae_rf = mean_absolute_error(Y_test, Y_pred_rf)
mse_rf = mean_squared_error(Y_test, Y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(Y_test, Y_pred_rf)

# Print evaluation metrics
print("Random Forest Regression Results:")
print("Mean Absolute Error (MAE):", mae_rf)
print("Mean Squared Error (MSE):", mse_rf)
print("Root Mean Squared Error (RMSE):", rmse_rf)
print("R-squared (R2):", r2_rf)

Random Forest Regression Results:
Mean Absolute Error (MAE): 0.03292421341767779
Mean Squared Error (MSE): 0.0018138508115986987
Root Mean Squared Error (RMSE): 0.04258932743773607
R-squared (R2): 0.6724508586044002


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

#random forest hyperparameter tuning, no categorical vars

X = combined_player_stats.iloc[1:, 5:-1].values
Y = combined_player_stats.iloc[1:, -1].values  # Only the last column, excluding first row

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the parameter grid
param_grid_rf = {
    'n_estimators': [150,200, 250],  # Number of trees in the forest
    'min_samples_split': [2,4,16]
}

# Define the model
model_rf = RandomForestRegressor(random_state=42)

# Initialize GridSearchCV
grid_search_rf = GridSearchCV(estimator=model_rf, param_grid=param_grid_rf, cv=5, scoring='neg_mean_absolute_error', verbose=1)

# Fit GridSearchCV to find the best hyperparameters
grid_search_rf.fit(X_train, Y_train)

# Get the best model and its hyperparameters
best_model_rf = grid_search_rf.best_estimator_
best_params_rf = grid_search_rf.best_params_

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params_rf)

# Predictions using the best model
Y_pred_rf_best = best_model_rf.predict(X_test)

# Compute evaluation metrics with the best model
mae_rf_best = mean_absolute_error(Y_test, Y_pred_rf_best)
mse_rf_best = mean_squared_error(Y_test, Y_pred_rf_best)
rmse_rf_best = np.sqrt(mse_rf_best)
r2_rf_best = r2_score(Y_test, Y_pred_rf_best)

# Print evaluation metrics with the best model
print("\nRandom Forest Regression Results with Best Hyperparameters:")
print("Mean Absolute Error (MAE):", mae_rf_best)
print("Mean Squared Error (MSE):", mse_rf_best)
print("Root Mean Squared Error (RMSE):", rmse_rf_best)
print("R-squared (R2):", r2_rf_best)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best Hyperparameters:
{'min_samples_split': 16, 'n_estimators': 250}

Random Forest Regression Results with Best Hyperparameters:
Mean Absolute Error (MAE): 0.03277518134438228
Mean Squared Error (MSE): 0.0017722038370401197
Root Mean Squared Error (RMSE): 0.042097551437585055
R-squared (R2): 0.679971560236065


In [13]:
#random forest with categorical variables
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

combined_player_stats['Squad'] = combined_player_stats['Squad'].astype(str)
combined_player_stats['Comp'] = combined_player_stats['Comp'].astype(str)
combined_player_stats['Nation'] = combined_player_stats['Nation'].astype(str)
#combined_player_stats['Pos'] = combined_player_stats['Pos'].astype(str)

# Splitting the data into input (X) and output (Y) variables
X = combined_player_stats.iloc[1:, :-1].drop(columns=['Player','Pos'])  # Exclude first row and 'Player' column
Y = combined_player_stats.iloc[1:, -1]   # Only the last column, excluding first row

# Define categorical columns
categorical_cols = ['Squad', 'Comp', 'Nation']

# Define the model
model_rf = RandomForestRegressor(n_estimators=250,min_samples_split=8,random_state=42)

# Define the column transformer including one-hot encoding for categorical columns
column_transformer_rf = ColumnTransformer(
    [('onehotencoder', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'
)

# Create a pipeline with column transformer and random forest regressor
pipeline_rf = Pipeline([
    ('preprocessor', column_transformer_rf),
    ('regressor', model_rf)
])

# Split the data into training and testing sets
X_train_rf, X_test_rf, Y_train_rf, Y_test_rf = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the pipeline
pipeline_rf.fit(X_train_rf, Y_train_rf)

# Predictions
Y_pred_rf = pipeline_rf.predict(X_test_rf)

# Compute evaluation metrics
mae_rf = mean_absolute_error(Y_test_rf, Y_pred_rf)
mse_rf = mean_squared_error(Y_test_rf, Y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(Y_test_rf, Y_pred_rf)

# Print evaluation metrics
print("Random Forest Regression Results:")
print("Mean Absolute Error (MAE):", mae_rf)
print("Mean Squared Error (MSE):", mse_rf)
print("Root Mean Squared Error (RMSE):", rmse_rf)
print("R-squared (R2):", r2_rf)

Random Forest Regression Results:
Mean Absolute Error (MAE): 0.030640576249188336
Mean Squared Error (MSE): 0.0016105503927859735
Root Mean Squared Error (RMSE): 0.04013166321978163
R-squared (R2): 0.7091632922850852
